# 서버와 동일한 VitalDB 데이터 다운로드 받는 법

VitalDB 데이터 셋 이용
본 예제에서는 오픈 생체 신호 데이터셋인 VitalDB를 이용하는 모든 사용자는 반드시 아래 Data Use Agreement에 동의하여야 합니다.

https://vitaldb.net/data-bank/?query=guide&documentId=13qqajnNZzkN7NZ9aXnaQ-47NWy7kx-a6gbrcEsi-gak&sectionId=h.usmoena3l4rb

동의하지 않을 경우 이 창을 닫으세요.

In [1]:
# vitalDB에서 다운로드 받기위해 필요한 파일
!wget -N https://raw.githubusercontent.com/vitaldb/vitalutils/master/python/vitaldb.py

--2021-08-03 11:17:57--  https://raw.githubusercontent.com/vitaldb/vitalutils/master/python/vitaldb.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19412 (19K) [text/plain]
Saving to: ‘vitaldb.py’

vitaldb.py          100%[===================>]  18.96K  --.-KB/s    in 0.005s  

Last-modified header missing -- time-stamps turned off.
2021-08-03 11:17:57 (3.85 MB/s) - ‘vitaldb.py’ saved [19412/19412]



In [2]:
import os
import sys
import numpy as np
import pandas as pd
import vitaldb
from tqdm import tqdm

In [3]:
# VitalDB Web API를 통해 데이터 로딩
df_trks = pd.read_csv('https://api.vitaldb.net/trks')
df_cases = pd.read_csv("https://api.vitaldb.net/cases")
df_cases['sex'] = (df_cases['sex'] == 'F').astype(int)  # female은 1으로, male 은 0으로 치환

# BIS, EEG1_WAV, EEG2_WAV, SEF, SQI 파일이 모두 존재하고 18세 이상인 피험자 ID 추출
caseids = list(set(df_trks[df_trks['tname'] == 'BIS/BIS']['caseid']) & \
set(df_trks[df_trks['tname'] == 'BIS/EEG1_WAV']['caseid']) & \
set(df_trks[df_trks['tname'] == 'BIS/EEG2_WAV']['caseid']) & \
set(df_trks[df_trks['tname'] == 'BIS/SEF']['caseid']) & \
set(df_trks[df_trks['tname'] == 'BIS/SQI']['caseid']) & \
set(df_cases[df_cases['age'] > 18]['caseid']))

print('total case: ', len(caseids))

total case:  5510


In [4]:
tnames = ['BIS/BIS', 'BIS/SQI', 'BIS/EEG1_WAV'] # 추출할 파라미터
interval = 1/100  # 100Hz

# 랜덤으로 선택한 5510명 중 1000명의 리스트
list_caseID = pd.read_csv('selectedCaseIDs.csv',header=None)

In [5]:
SAVEPATH = '../data/vitaldb/raw'

# SAVEPATH가 없을 시 폴더 생성
try:
    os.makedirs(SAVEPATH)
except OSError as exc:
    print('error')
    # if exc.errno != errno.EEXIST:
    #     raise
    pass

for idx, sbjID in enumerate(tqdm(list_caseID.to_numpy())):
#     if a == 10: break   # 10명만 다운받음. 이 줄 주석처리하면 리스트에 적힌 1000명 다운로드
    vals = vitaldb.load_case(tnames, sbjID[0], interval)    # vitalDB에서 다운로드
    np.save(os.path.join(SAVEPATH, f'{idx}.npy'), vals)     # SAVEPATH에 저장

100%|██████████| 1000/1000 [12:20<00:00,  1.35it/s]
